# 2개의 층

In [18]:
# 케라스 API 를 사용해서 패션 MNIST 데이터셋을 로드
from tensorflow import keras
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data() # 책과는 연산식 부분이 약간 다름

이미지를 로드한 다음  
픽셀값을 0 ~ 255 범위에서 0 ~ 1 사이로 변환하고, 28 * 28 크기의  
2차원 배열을 784 크기의 1차원 배열로 펼친다.  
마지막으로 사이킷런의 train_test_split()함수로 훈련세트와 검증세트를 분할  


In [19]:
from sklearn.model_selection import train_test_split
train_scaled = train_input / 255.0
train_scaled = train_scaled.reshape(-1, 28 * 28)
train_scaled, val_scaled, train_target, val_target = train_test_split(
    train_scaled, train_target, test_size = 0.2, random_state = 42
)

이제 인공신경망 모델에 층을 2개 추가한다.  
전에 만든 것과 차이점은 입력층과 출력층 사이에 밀집층이 추가되는 것이다.  
이러한 입력층 ~ 출력층 사이의 모든 층을 <b>은닉층</b>이라고 한다.  
은닉층에는 활성화 함수가 있다.  
  

출력층에 적용하는 함수는 다음으로 제한된다.  
이진분류일 경우 시그모이드 함수를 사용하고  
다중분류일 경우 소프트맥스 함수를 사용한다.  
  
하지만 은닉층의 활성화 함수는 비교적 자유롭다.  
주로 시그모이드 함수와 볼 렐루 함수 등을 사용한다.  
  
시그모이드 함수는 뉴런의 출력 z값을 0과 1사이로 압축한다.  
  
  

---
시그모이드 함수를 사용한 은닉층과  
소프트맥스 함수를 사용한 출력층을  
각각의 Dense 클래스로 만들어보자  

In [20]:
dense1 = keras.layers.Dense(100, activation = 'sigmoid', input_shape=(784,))  # 은닉층의 뉴런의 수를 정하는 것에는 특별한 기준이 없다. 즉 경험에 의존해야 한다.
dense2 = keras.layers.Dense(10, activation = 'softmax')

dense1은 은닉층이고 100개의 뉴런을 가진 밀집층이다. 활성화 함수로 시그모이드를 사용하고 입력의 크기를 784로 지정했다.  
dense2는 출력층이고  10개의 뉴런을 가졌고 소프트맥스 함수를 활성함수로 지정했다.  


# 심층 신경망 만들기
이제 dense1과 dense2를 Sequential 클래스에 추가해서 심층 신경망을 만들어보자


In [21]:
model = keras.Sequential([dense1, dense2])

dense1과 dense2를 리스트로 만들어 전달한다.  
출력층은 가장 마지막에 두어야 한다.  

---
인공신경망의 강력한 성능은 이렇게 층을 추가하여 입력 데이터에 대해 연속적인 학습을 진행하는 능력에서 나온다.  
케라스는 모델의 summary() 메서드를 호출하면 층에 대한 유용한 정보를 얻을 수 있다.


In [22]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 100)               78500     
                                                                 
 dense_7 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


모델이름 sequential  
층의 이름을 지정하지 않으면 dense로 초기화  
출력크기는 (None, 100)  
여기서 None은 샘플 개수가 정의되어 있지 않기 때문에 None이며  
은닉층의 뉴런개수가 100개이므로 100개의 출력이 나온다.  
즉, 샘플마다 784개의 픽셀값이 은닉층을 통과하면서 100개의 특성으로 압축되었다.  
파라미터는 78500개다.  
이는 784개의 픽셀 * 100개의 뉴런 + 100개의 절편(뉴런마다 1개)으로 이루어진다.  
이는 784개의 픽셀과 100개의 모든 조합에 대한 가중치다.  

---

두 번째 층의 출력 크기는(None, 10)  
즉, 뉴런 개수가 10개다.  
파라미터는 100개의 은닉뉴런층 * 10개의 출력층 뉴런 + 출력층의 뉴런마다 하나의 절편(10) 이므로  
100 * 10 + 10 이므로 1010개이다.

---

summary() 메서드의 마지막에는  
총 모델 파라미터 개수(78500 + 1010)와  
훈련되는 파라미터 수가 나온다.  
그 아래 Non-trainable params는 훈련되지 않은 파라미터의 수다.  


# 층을 추가하는 다른 방법
앞에서는 Dense 클래스의 객체 dense1, dense2를 만들어 sequential 클래스에 전달했다.  
두 객체를 따로 저장해서 쓸 일이 없기에 Sequential 클래스 생성자 안에 Dense 클래스 객체를 만드는 경우가 많았다.

## 방법 1

In [23]:
model = keras.Sequential([keras.layers.Dense(100, activation='sigmoid', input_shape=(784,), name='hidden'), keras.layers.Dense(10, activation = 'softmax', name='output')], name='패션 MNIST 모델')

In [24]:
model.summary()

Model: "패션 MNIST 모델"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 hidden (Dense)              (None, 100)               78500     
                                                                 
 output (Dense)              (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


위 방법은 추가되는 층을 한 눈에 쉽게 볼 수 있다는 장점이 있다.

# 방법 2

In [25]:
model = keras.Sequential()
model.add(keras.layers.Dense(100, activation = 'sigmoid', input_shape=(784,)))
model.add(keras.layers.Dense(10, activation = 'softmax'))

In [26]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               78500     
                                                                 
 dense_9 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


Sequential 클래스의 객체를 만들고 이 객체의 add() 메서드를 호출하여 층을 추가하는 방식이다.  


---

In [27]:
# 모델 훈련, 에포크는 5번
model.compile(loss='sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.5661 - accuracy: 0.8056
Epoch 2/5
1500/1500 [==============================] - 9s 6ms/step - loss: 0.4118 - accuracy: 0.8505
Epoch 3/5
1500/1500 [==============================] - 8s 5ms/step - loss: 0.3772 - accuracy: 0.8646
Epoch 4/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3532 - accuracy: 0.8730
Epoch 5/5
1500/1500 [==============================] - 5s 4ms/step - loss: 0.3359 - accuracy: 0.8767


추가된 층이 성능을 향상시켰다.

# 렐루 함수
시그모이드 함수는 그래프를 보면 왼쪽과 오른쪽 끝으로 갈 수록 그래프가 누워버리기 때문에  
올바른 출력을 만드는데 신속하게 대응하지 못한다.  
특히 층이 많을 수록 그 효과가 누적되에 학습을 어렵게 한다.  
그래서 만들어진 것이 바로 렐루 함수다.  
렐루 함수는 입력이 양수면 그냥 입력을 통과시키고 음수일 경우에는 0으로 만든다.  


In [28]:
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation = 'relu'))
model.add(keras.layers.Dense(10, activation = 'softmax'))

첫 번째 Dense 층에 있던 input_shape 매개변수를 Flatten층으로 옮긴다.  
첫 번째 Dense 층의 활성화 함수를 relu로 바꿨다.  

In [29]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_10 (Dense)            (None, 100)               78500     
                                                                 
 dense_11 (Dense)            (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________


flatten의 파라미터는 0개인데 이는 학습하는 층이 아니라서 그렇다.  
---
훈련데이터를 다시 준비해서 모델을 훈련해보자  
reshape() 메서드를 적용하지 않았다.  

In [30]:
(train_input, train_target), (test_input, test_target) = keras.datasets.fashion_mnist.load_data()
train_scaled = train_input / 255.0
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state = 42)

In [31]:
model.compile(loss='sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.5279 - accuracy: 0.8131
Epoch 2/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3917 - accuracy: 0.8590
Epoch 3/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3543 - accuracy: 0.8725
Epoch 4/5
1500/1500 [==============================] - 5s 3ms/step - loss: 0.3315 - accuracy: 0.8806
Epoch 5/5
1500/1500 [==============================] - 4s 3ms/step - loss: 0.3138 - accuracy: 0.8870


시그모이드 함수를 사용했을 때와 비교하면 성능이 조금 향상되었다.

In [32]:
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3736 - accuracy: 0.8746


[0.37357133626937866, 0.8745833039283752]

# 옵티마이저
하이퍼파라미터는 모델이 학습하지 않아 사람이 지정해주어야 하는 파라미터를 말한다.  
신경망에는 특히 하이퍼파라미터가 많다.  
여러개의 은닉층을 추가할 수도 있지만 추가할 은닉층의 개수는 하이퍼파라미터다.  
은닉층의 뉴런 개수도 하이퍼파라미터다.  
활성화 함수도 우리가 선택할 하이퍼파라미터중 하나다.  
심지어 층의 종류도 하이퍼파라미터다.  
epochs의 매개변수 하이퍼파라미터다.  
  
---  
케라스가 제공하는 다양한 종류의 경사하강법 알고리즘을 옵티마이저라고 부른다.  
아래와 같이 사용한다.  
model.compile(optimizer='sgd', loss = 'sparse_categorical_crossentropy', metrics ='accuracy')  

---
기본 경사 하강법 옵티마이저는 모두 SHD 클래스에서 제공한다.  
SGD 클래스의 nesterov 매개변수를 True로 바꾸면 네스테로프 모멘텀 최적화를 사용한다.  
대부분의 경우, 네스테로프 모멘텀이 기본 확률적 경사 하강법보다 성능이 좋다.  

---
모델이 최적점에 가까이 가면 학습률을 낮춘다.  
이렇게 하면 안정적으로 최적점에 수렴하는데 적응적 학습률이라고 한다.  
적응적 학습률을 사용하는 대표적인 옵티마이저는 Adagrad와 RMSprop이다.  

In [33]:
# Adam 클래스의 매개변수 기본값으로 패션MNIST를 훈련해보자
model = keras.Sequential()
model.add(keras.layers.Flatten(input_shape=(28, 28)))
model.add(keras.layers.Dense(100, activation='relu'))
model.add(keras.layers.Dense(10, activation ='softmax'))

In [36]:
# 컴파일 메서드의 옵티마이저를 아담으로 설정하고 5번의 에포크 동안 훈련
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = 'accuracy')
model.fit(train_scaled, train_target, epochs = 5)

Epoch 1/5
1500/1500 [==============================] - 6s 3ms/step - loss: 0.5274 - accuracy: 0.8156
Epoch 2/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3916 - accuracy: 0.8605
Epoch 3/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3536 - accuracy: 0.8718
Epoch 4/5
1500/1500 [==============================] - 7s 5ms/step - loss: 0.3276 - accuracy: 0.8807
Epoch 5/5
1500/1500 [==============================] - 6s 4ms/step - loss: 0.3096 - accuracy: 0.8868


RMSprop를 사용했을 때와 거의 같은 성능을 보여준다.

In [37]:
# 성능 확인
model.evaluate(val_scaled, val_target)

375/375 [==============================] - 1s 2ms/step - loss: 0.3246 - accuracy: 0.8814


[0.32462435960769653, 0.8814166784286499]

기본 RMSprop보다 조금 더 나은 성능이 나왔다.

# 케라스 API를 활용한 심층 신경망
여러 개의 층을 추가해 다층 인공 신경망을 만들어 보았다.  
이런 인공 신경망을 심층 신경망이라고 부른다.  
또 케라스 API를 통해 층을 추가하는 여러가지 방법을 알아보았다.  

# 정리
### 심층신경망
2개 이상의 층을 포함한 신경망, 다층 인공신경망, 딥러닝 이라고도 한다.  
### 렐루 함수
이미지 분류 모델의 은닉층에서 많이 사용하는 활성화 함수.  
시그모이드 함수와 달리 층이 많아져도 학습곤란 문제가 발생하지 않는다.
### 옵티마이저
신경망의 가중치와 절편을 학습하기 위한 알고리즘 또는 방법  
케라스에는 다양한 경사하강법 알고리즘이 있는데  
대표적으로 SGD, 네스테로프모멘텀, RMSprop, Adam 등이다.